In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

conn = sqlite3.connect("fitness_app.db")

In [ ]:
import os

os.makedirs("plots", exist_ok=True)

In [ ]:
funnel_query = """
WITH step1 AS (
    SELECT DISTINCT user_id
    FROM events
    WHERE event_type = 'account_created'
),
step2 AS (
    SELECT DISTINCT e.user_id
    FROM events e
    JOIN step1 s ON e.user_id = s.user_id
    WHERE e.event_type = 'profile_setup'
),
step3 AS (
    SELECT DISTINCT e.user_id
    FROM events e
    JOIN step2 s ON e.user_id = s.user_id
    WHERE e.event_type = 'first_workout'
),
step4 AS (
    SELECT DISTINCT e.user_id
    FROM events e
    JOIN step3 s ON e.user_id = s.user_id
    WHERE e.event_type = 'week1_return'
)
SELECT
   (SELECT COUNT(*) FROM step1) AS account_created,
   (SELECT COUNT(*) FROM step2) AS profile_setup,
   (SELECT COUNT(*) FROM step3) AS first_workout,
   (SELECT COUNT(*) FROM step4) AS week1_return;
"""

funnel_df = pd.read_sql(funnel_query, conn).T.reset_index()
funnel_df.columns = ["event_type", "users"]

Drop-Off Calculations

In [ ]:
funnel_df["previous_stage"] = funnel_df["users"].shift(1)
funnel_df["drop_off_users"] = funnel_df["previous_stage"] - funnel_df["users"]
funnel_df["drop_off_percent"] = (funnel_df["drop_off_users"] / funnel_df["previous_stage"] * 100).round(2)
funnel_df.fillna(0, inplace=True)

print(funnel_df)

Funnel Visualization

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x="users", y="event_type", data=funnel_df, palette="Blues_d")

for i, (users, drop) in enumerate(zip(funnel_df["users"], funnel_df["drop_off_percent"])):
    if i > 0:
        plt.text(users, i, f"{drop:.1f}%", va="center", ha="left", fontsize=10, color="red")

plt.title("User Funnel with Drop-Offs")
plt.xlabel("Number of Users")
plt.ylabel("Stage")
plt.savefig("plots/funnel_chart.png", dpi=300, bbox_inches="tight")
plt.show()

Segment Analysis

In [ ]:
segment_query = """
WITH step1 AS (
    SELECT DISTINCT user_id, acquisition_channel
    FROM events
    JOIN users USING(user_id)
    WHERE event_type = 'account_created'
),
step2 AS (
    SELECT DISTINCT e.user_id, u.acquisition_channel
    FROM events e
    JOIN step1 u ON e.user_id = u.user_id
    WHERE e.event_type = 'profile_setup'
),
step3 AS (
    SELECT DISTINCT e.user_id, u.acquisition_channel
    FROM events e
    JOIN step2 u ON e.user_id = u.user_id
    WHERE e.event_type = 'first_workout'
),
step4 AS (
    SELECT DISTINCT e.user_id, u.acquisition_channel
    FROM events e
    JOIN step3 u ON e.user_id = u.user_id
    WHERE e.event_type = 'week1_return'
)
SELECT
    acquisition_channel,
    (SELECT COUNT(*) FROM step1 s WHERE s.acquisition_channel = u.acquisition_channel) AS account_created,
    (SELECT COUNT(*) FROM step2 s WHERE s.acquisition_channel = u.acquisition_channel) AS profile_setup,
    (SELECT COUNT(*) FROM step3 s WHERE s.acquisition_channel = u.acquisition_channel) AS first_workout,
    (SELECT COUNT(*) FROM step4 s WHERE s.acquisition_channel = u.acquisition_channel) AS week1_return
FROM step1 u
GROUP BY acquisition_channel;
"""

segment_df = pd.read_sql(segment_query, conn)
print(segment_df.head())

Cohort Retention

In [ ]:
cohort_query = """
WITH UserCohorts AS (
    SELECT
        user_id,
        MIN(event_timestamp) as cohort_date
    FROM events
    WHERE event_type = 'account_created'
    GROUP BY user_id
),
ActivityByMonth AS (
    SELECT
        T1.user_id,
        STRFTIME('%Y-%m', T1.cohort_date) as cohort_month,
        (STRFTIME('%Y', T2.event_timestamp) - STRFTIME('%Y', T1.cohort_date)) * 12 +
        (STRFTIME('%m', T2.event_timestamp) - STRFTIME('%m', T1.cohort_date)) AS retention_month
    FROM UserCohorts AS T1
    JOIN events AS T2 ON T1.user_id = T2.user_id
)
SELECT
    cohort_month,
    retention_month,
    COUNT(DISTINCT user_id) as users
FROM ActivityByMonth
GROUP BY cohort_month, retention_month
ORDER BY cohort_month, retention_month;
"""

cohort_df = pd.read_sql(cohort_query, conn)
print(cohort_df.head())

In [ ]:
query = """
WITH UserCohorts AS (
    SELECT
        user_id,
        MIN(event_timestamp) as cohort_date
    FROM events
    GROUP BY user_id
),
ActivityByMonth AS (
    SELECT
        T1.user_id,
        STRFTIME('%Y-%m', T1.cohort_date) as cohort_month,
        (STRFTIME('%Y', T2.event_timestamp) - STRFTIME('%Y', T1.cohort_date)) * 12 +
        (STRFTIME('%m', T2.event_timestamp) - STRFTIME('%m', T1.cohort_date)) AS retention_period_months
    FROM UserCohorts AS T1
    JOIN events AS T2 ON T1.user_id = T2.user_id
),
AggregatedData AS (
    SELECT
        cohort_month,
        retention_period_months,
        COUNT(DISTINCT user_id) as users
    FROM ActivityByMonth
    GROUP BY cohort_month, retention_period_months
)
SELECT
    cohort_month,
    retention_period_months,
    users,
    MAX(users) OVER (PARTITION BY cohort_month) AS total_cohort_users,
    ROUND(100.0 * users / MAX(users) OVER (PARTITION BY cohort_month), 2) AS retention_rate
FROM AggregatedData
ORDER BY cohort_month, retention_period_months;
"""
cohort_df = pd.read_sql(query, conn)
cohort_df.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=cohort_df,
    x="retention_period_months",
    y="retention_rate",
    hue="cohort_month",
    marker="o"
)
plt.title("Cohort Retention Over Time (%)", fontsize=14)
plt.xlabel("Months Since Signup")
plt.ylabel("Retention Rate (%)")
plt.legend(title="Cohort Month", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(alpha=0.3)
plt.savefig("plots/cohort_retention_line.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

for cohort, group in cohort_df.groupby("cohort_month"):
    plt.fill_between(group["retention_period_months"], group["users"], alpha=0.3)
    plt.plot(group["retention_period_months"], group["users"], marker="o", label=cohort)

plt.title("Cohort Retention (Area Curves)", fontsize=14)
plt.xlabel("Months Since Signup")
plt.ylabel("Users Retained")
plt.legend(title="Cohort Month", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.savefig("plots/cohort_retention_area.png", dpi=300, bbox_inches="tight")
plt.show()

Statistical Analysis

In [ ]:
contingency = []
for i in range(1, len(funnel_df)):
    success = funnel_df.loc[i, "users"]
    fail = funnel_df.loc[i-1, "users"] - success
    contingency.append([success, fail])

chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-Square Test Statistic:", chi2)
print("p-value:", p)